# Import thư viện

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models



# ⚙️ Cấu hình
IMG_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 50
AUTOTUNE = tf.data.AUTOTUNE
image_path = '/kaggle/input/btxrd-final1/images'
mask_path = '/kaggle/input/btxrd-final1/masks'
df = pd.read_excel('/kaggle/input/btxrd-final1/classification.xlsx')

# 🧹 Chuẩn bị DataFrame
df = df[df['mask_flag'] == 1]
df_train = df[df['tumor_category'] == 1].copy()
df_valid = df[df['tumor_category'] == 3].copy()
df_test  = df[df['tumor_category'] == 2].copy()

for d in [df_train, df_valid, df_test]:
    d['image_path'] = d['image_id'].apply(lambda x: os.path.join(image_path, f"{os.path.splitext(x)[0]}.jpg"))
    d['mask_path']  = d['image_id'].apply(lambda x: os.path.join(mask_path, f"{os.path.splitext(x)[0]}.png"))


2025-05-17 10:44:36.527175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747478676.727338      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747478676.787546      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Tiền xử lý dữ liệu và tạo tf.data.Dataset


In [2]:
def load_image_and_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, IMG_SIZE)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.image.resize(mask, IMG_SIZE)

    return image, mask

def build_dataset(df, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((df['image_path'].values, df['mask_path'].values))
    dataset = dataset.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(512)
    dataset = dataset.batch(BATCH_SIZE).cache().prefetch(AUTOTUNE)
    return dataset

train_dataset = build_dataset(df_train, shuffle=True)
val_dataset   = build_dataset(df_valid)
test_dataset  = build_dataset(df_test)


I0000 00:00:1747478690.626039      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


# Mô hình U-Net

In [3]:
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def unet_model(input_shape=(256, 256, 1), num_classes=1):
    inputs = layers.Input(shape=input_shape)
    c1 = conv_block(inputs, 64); p1 = layers.MaxPooling2D()(c1)
    c2 = conv_block(p1, 128); p2 = layers.MaxPooling2D()(c2)
    c3 = conv_block(p2, 256); p3 = layers.MaxPooling2D()(c3)
    c4 = conv_block(p3, 512); p4 = layers.MaxPooling2D()(c4)
    bn = conv_block(p4, 1024)
    u1 = layers.UpSampling2D()(bn); u1 = layers.Concatenate()([u1, c4]); c5 = conv_block(u1, 512)
    u2 = layers.UpSampling2D()(c5); u2 = layers.Concatenate()([u2, c3]); c6 = conv_block(u2, 256)
    u3 = layers.UpSampling2D()(c6); u3 = layers.Concatenate()([u3, c2]); c7 = conv_block(u3, 128)
    u4 = layers.UpSampling2D()(c7); u4 = layers.Concatenate()([u4, c1]); c8 = conv_block(u4, 64)
    outputs = layers.Conv2D(num_classes, 1, activation='sigmoid', dtype='float32')(c8)
    return models.Model(inputs, outputs)

# Hàm loss Dice + Binary Crossentropy

In [4]:
def dice_loss(y_true, y_pred, smooth=1):
    y_true_f = tf.reshape(tf.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def combined_loss(y_true, y_pred, alpha=0.6):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return alpha * bce + (1 - alpha) * dice_loss(y_true, y_pred)

# Compile & Huấn luyện

In [ ]:
model = unet_model()
model.compile(optimizer='adam',loss=combined_loss,metrics=['accuracy'])
model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
                     tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)])

#model.save("unet.h5")

Epoch 1/50


I0000 00:00:1747478711.260689      91 service.cc:148] XLA service 0x7c7a84003120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747478711.261617      91 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747478713.046648      91 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747478739.879911      91 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


164/165 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.9480 - loss: 0.4962

E0000 00:00:1747478779.920939      92 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747478780.178086      92 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


165/165 ━━━━━━━━━━━━━━━━━━━━ 108s 369ms/step - accuracy: 0.9483 - loss: 0.4956 - val_accuracy: 0.0371 - val_loss: 4.9750 - learning_rate: 0.0010
Epoch 2/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 39s 236ms/step - accuracy: 0.9756 - loss: 0.4063 - val_accuracy: 0.9806 - val_loss: 0.4420 - learning_rate: 0.0010
Epoch 3/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 39s 236ms/step - accuracy: 0.9738 - loss: 0.4001 - val_accuracy: 0.8310 - val_loss: 0.5423 - learning_rate: 0.0010
Epoch 4/50
 63/165 ━━━━━━━━━━━━━━━━━━━━ 23s 227ms/step - accuracy: 0.9704 - loss: 0.3908

# Tính Dice Score trung bình

In [ ]:
# 📈 Đánh giá Dice từng ảnh
def evaluate_dice_per_image(model, dataset, threshold=0.3):
    dice_scores = []
    for images, masks in dataset:
        preds = model.predict(images)
        preds_bin = (preds > threshold).astype(np.float32)
        for i in range(len(images)):
            y_true = tf.reshape(tf.cast(masks[i], tf.float32), [-1])
            y_pred = tf.reshape(tf.cast(preds_bin[i], tf.float32), [-1])
            inter = tf.reduce_sum(y_true * y_pred)
            score = (2. * inter + 1) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1)
            dice_scores.append(score.numpy())
    return dice_scores

dice_scores = evaluate_dice_per_image(model, test_dataset)
print("📊 Số ảnh test:", len(dice_scores))
print(f"🎯 Dice trung bình: {np.mean(dice_scores):.4f}")
print(f"📉 Dice thấp nhất: {np.min(dice_scores):.4f}")
print(f"📈 Dice cao nhất:  {np.max(dice_scores):.4f}")

# Hiển thị ảnh đầu vào, mask và dự đoán

In [ ]:
def show_predictions(model, dataset, threshold=0.3, num_samples=3):
    for images, masks in dataset.take(1):
        preds = model.predict(images)
        preds_bin = preds > threshold

        plt.figure(figsize=(16, 5 * num_samples))
        for i in range(num_samples):
            plt.subplot(num_samples, 3, i * 3 + 1)
            plt.imshow(tf.squeeze(images[i]), cmap='gray')
            plt.title("Input Image")
            plt.axis('off')

            plt.subplot(num_samples, 3, i * 3 + 2)
            plt.imshow(tf.squeeze(masks[i]), cmap='gray')
            plt.title("Ground Truth Mask")
            plt.axis('off')

            plt.subplot(num_samples, 3, i * 3 + 3)
            plt.imshow(tf.squeeze(preds_bin[i]), cmap='gray')
            plt.title("Predicted Mask")
            plt.axis('off')

        plt.tight_layout()
        plt.show()

# 🔍 Gọi hiển thị sau khi train xong hoặc load model
show_predictions(model, test_dataset, threshold=0.3, num_samples=3)


# Logging với WandB